In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm
import seaborn as sns

%matplotlib qt

# matplotlib style = BLACK BACKGROUND, WHITE TEXT and AXES

# reset to default
plt.rcdefaults()
plt.style.use('dark_background')
plt.rcParams['axes.facecolor'] = 'black'
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'

In [ ]:
simulation_props = {
    'duration': 30*60, # 30 minutes
    'fps': 30, # frames per second
}

arena_props = {
    'radius': 75, # mm
    'center': (0, 0), # center of the arena
}

trail_props = {
    'graded': False,
    'trail_radius': 50, # mm
    'trail_width': 3, # mm
    'trail_center': (0, 0), # center of the trail
    'peak_concentration': 1, # peak concentration of the trail
    'peak_distance': 0, # distance from the center of the trail where the peak concentration is first reached
}

def trail_odor(x,y,t,trail_props):
    """
    Generate the odor concentration at a given position and time
    x: x position (mm)
    y: y position (mm)
    t: time (s)
    trail_props: properties of the trail
    """
    # calculate the distance from the center of the trail
    distance = np.sqrt((x - trail_props['trail_center'][0])**2 + (y - trail_props['trail_center'][1])**2) - trail_props['trail_radius']
    
    # if the distance is within trail_width, return 1
    if np.abs(distance) <= trail_props['trail_width']/2:
        if trail_props['graded']:
            # interpolate between 0 and peak_concentration based on the peak_distance
            position = np.clip(trail_props['trail_width']/2 - np.abs(distance)/(trail_props['trail_width']/2-trail_props['peak_distance']), 0, 1)
            return trail_props['peak_concentration'] * position
        else:
            return trail_props['peak_concentration']
    else:
        return 0
    
fly_props = {
    # Kinematic properties
    'walking_speed': 15, # mm/s (speed of the walking state of the fly)
    'rotation_diffusion': np.deg2rad(0.22), # rad/s (sd of rotation diffusion)
    'turn_rate': 1.0, # Hz (rate of turning in the turning state of the fly)
    'turn_magnitude_range': np.deg2rad((8,30)), # rad (range of turn magnitudes in the turning state of the fly)
    'rate_stop_to_walk': 0.5, # Hz (rate of transition from the stop state to the walking state)
    'rate_walk_to_stop': 0.05, # Hz (rate of transition from the walking state to the stop state)
    
    # Odor integration properties
    'odor_integration_timescale': 1/simulation_props['fps'], # s (timescale of odor integration)
    'odor_magnitude': 1, # (magnitude of the odor signal)


    'asymmetry_factor': 20, # (increase in turn rate when odor asymmetry is detected)


    


walking_speed=10 # mm/s (speed of the walking state of the fly)
rotation_diffusion=np.deg2rad(0.22) # rad/s (sd of rotation diffusion)
turn_rate=1.0 # Hz (rate of turning in the turning state of the fly)
asymmetry_factor=20 # (increase in turn rate when odor asymmetry is detected)
turn_magnitude_range= np.deg2rad((8,30)) # rad (range of turn magnitudes in the turning state of the fly)
rate_stop_to_walk=0.5 # Hz (rate of transition from the stop state to the walking state)
rate_walk_to_stop=0.05 # Hz (rate of transition from the walking state to the stop state)

odor_integration_timescale=1/fps # s (timescale of odor integration)
odor_magnitude=1 # (magnitude of the odor signal)

# antennal distance
antennal_distance=1 # mm
    


In [ ]:
def get_odor_concentration(x, y, t, odor_generator, generator_props):
    """
    Get the odor concentration at a given position and time
    x: x position (mm)
    y: y position (mm)
    t: time (s)
    odor_generator: function that generates the odor concentration at a given position and time
    """
    return odor_generator(x, y, t, generator_props)

In [ ]:
T=30*60 # s (duration of the simulation)
fps=60 # Hz (frame rate of the simulation)
arena_radius=75 # mm (radius of the arena)

trail_radius=50 # mm (radius of the trail)
trail_width=5 # mm (width of the trail)

walking_speed=10 # mm/s (speed of the walking state of the fly)
rotation_diffusion=np.deg2rad(0.22) # rad/s (sd of rotation diffusion)
turn_rate=1.0 # Hz (rate of turning in the turning state of the fly)
asymmetry_factor=20 # (increase in turn rate when odor asymmetry is detected)
turn_magnitude_range= np.deg2rad((8,30)) # rad (range of turn magnitudes in the turning state of the fly)
rate_stop_to_walk=0.5 # Hz (rate of transition from the stop state to the walking state)
rate_walk_to_stop=0.05 # Hz (rate of transition from the walking state to the stop state)

odor_integration_timescale=1/fps # s (timescale of odor integration)
odor_magnitude=1 # (magnitude of the odor signal)

# antennal distance
antennal_distance=1 # mm

# PROCESSING
trail_start_radius=trail_radius-trail_width/2
trail_end_radius=trail_radius+trail_width/2

# get odor function
def get_odor(x,y):
    # x,y are the position of the fly in mm
    # return the odor concentration at that position
    if np.sqrt(x**2+y**2)>trail_start_radius and np.sqrt(x**2+y**2)<trail_end_radius:
        return 1
    return 0

# ASSUME ALL PROCESSES ARE POISSON
turn_rate=turn_rate/fps # 1/frame (rate of turning in the turning state of the fly)
asymmetry_factor=asymmetry_factor/fps # 1/frame (increase in turn rate when odor asymmetry is detected)
rate_stop_to_walk=rate_stop_to_walk/fps # 1/frame (rate of transition from the stop state to the walking state)
rate_walk_to_stop=rate_walk_to_stop/fps # 1/frame (rate of transition from the walking state to the stop state)

odor_integration_timescale=odor_integration_timescale*fps # frames (timescale of odor integration)

walking_distance=walking_speed/fps # mm/frame (distance covered by the fly in one frame in the walking state)
N=int(T*fps) # number of frames in the simulation
state = np.zeros(N) # state of the fly (0=stop, 1=walking)
heading = np.zeros(N) # heading of the fly (rad)
x = np.zeros(N) # x position of the fly (mm)
y = np.zeros(N) # y position of the fly (mm)
odors = np.zeros((N,2)) # odor concentration at the left and right antennae


heading[0]=np.random.rand()*2*np.pi # initial heading
at_wall=False
odor_asymmetry_seen=False

for i in tqdm(range(1,N)):
    # update state
    if state[i-1]==0: # in the stop state
        if np.random.rand()<rate_stop_to_walk: # transition to walking state
            state[i]=1
        else:
            state[i]=0
    else:
        if np.random.rand()<rate_walk_to_stop: # transition to stop state
            state[i]=0
        else:
            state[i]=1
    
    # get left and right antennal positions
    left_x=x[i-1]+np.cos(heading[i-1]+np.pi/2)*antennal_distance
    left_y=y[i-1]+np.sin(heading[i-1]+np.pi/2)*antennal_distance
    right_x=x[i-1]+np.cos(heading[i-1]-np.pi/2)*antennal_distance
    right_y=y[i-1]+np.sin(heading[i-1]-np.pi/2)*antennal_distance
    odor_left=get_odor(left_x,left_y)
    odor_right=get_odor(right_x,right_y)
    
    # update odor estimate assuming decay process
    odors[i,0]=odors[i-1,0]+(odor_left*odor_magnitude-odors[i-1,0])/odor_integration_timescale
    odors[i,1]=odors[i-1,1]+(odor_right*odor_magnitude-odors[i-1,1])/odor_integration_timescale

    # update position based on state
    if state[i]==0: # in the stop state
        x[i]=x[i-1]
        y[i]=y[i-1]
        heading[i]=heading[i-1]+np.random.randn()*rotation_diffusion
    else: # in the walking state
        # sample a turn
        if odor_left>odor_right:
            turn_direction=1
            odor_asymmetry_seen=True
            # print('left')
        elif odor_right>odor_left:
            turn_direction=-1
            odor_asymmetry_seen=True
            # print('right')
        else:
            turn_direction=np.random.choice([-1,1])
            odor_asymmetry_seen=False

        if at_wall: # if fly is at the wall, it must turn
            heading[i]=heading[i-1]+ np.random.uniform(*turn_magnitude_range)*turn_direction + np.random.randn()*rotation_diffusion
            at_wall=False
        elif np.random.rand()<turn_rate+np.abs(odor_right-odor_left)*asymmetry_factor: # turn
            heading[i]=heading[i-1]+ np.random.uniform(*turn_magnitude_range)*turn_direction + np.random.randn()*rotation_diffusion
        else:
            heading[i]=heading[i-1]+np.random.randn()*rotation_diffusion
        # move forward
        new_x=x[i-1]+np.cos(heading[i])*walking_distance
        new_y=y[i-1]+np.sin(heading[i])*walking_distance
        # check if fly is outside the arena
        if np.sqrt(new_x**2+new_y**2)>arena_radius:
            # animal stops at the wall
            x[i]=x[i-1]
            y[i]=y[i-1]
            at_wall=True
        else:
            x[i]=new_x
            y[i]=new_y

fig, ax = plt.subplots()
# plot the trajectory
ax.plot(x,y)
# plot the arena
circle=plt.Circle((0,0),arena_radius,fill=False)
ax.add_artist(circle)
# plot the trail
circle=plt.Circle((0,0),trail_start_radius,fill=False,color='red')
ax.add_artist(circle)
circle=plt.Circle((0,0),trail_end_radius,fill=False,color='red')
ax.add_artist(circle)
# set the aspect of the plot to be equal
ax.set_aspect('equal')
plt.show()

# plot the odor concentration
plt.figure(figsize=(10,3))
plt.plot(odors)
# also plot the difference in odor concentration on a twinx axis
plt.twinx()
plt.plot(odors[:,0]-odors[:,1],'k--')
plt.ylabel('odor difference')
plt.legend(['left','right'])
plt.show()
